# Kaplan–Meier plots

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from lifelines import KaplanMeierFitter
from matplotlib.pyplot import figure
from lifelines.statistics import logrank_test

In [2]:
def Surv(counts):
    T = []; E = []; T1 = []; T0 = []; E1 = []; E0 = []
    counts = np.trim_zeros(counts, trim='f')
    deaths = counts[:-1]-counts[1:]
    start = counts[0]
    T1 = [[i+1]*deaths[i] for i in range(len(deaths)) if deaths[i]>0]
    T1 = [item for sublist in T1 for item in sublist]
    E1 = [1]*len(T1)
    T0 = [(len(counts)-1) for i in range(start-deaths.sum())]
    E0 = [0]*len(T0)
    T1.extend(T0); T = T1
    E1.extend(E0); E = E1
    return ({'T':T, 'E':E})
def Survs(countss):
    T = []; E = []    
    for counts in countss:
        surv = Surv(counts)
        T = np.concatenate((T, surv['T']))
        E = np.concatenate((E, surv['E']))
    return ({'T':T.astype(np.int8).tolist(), 'E':E.astype(np.int8).tolist()})

# Load lifespan

In [3]:
lifespan = pd.read_csv('fruitfly_lifespan.tsv', delimiter='\t'); print(lifespan.shape); lifespan = lifespan[lifespan['dont_use']==0]; print(lifespan.shape)
print(lifespan.columns)
lifespan = lifespan[['camera', 'position', 'vial', 'condition', 'concentration', 'compound', 'repetition', 'replicate', 'date_of_birth', 'start_flies', 'dont_use',
                     '08/07/2021', '16/07/2021', '22/07/2021', '29/07/2021', '05/08/2021', '12/08/2021', '19/08/2021', '26/08/2021', '02/09/2021', '09/09/2021', '16/09/2021', 
                     '23/09/2021', '30/09/2021', '07/10/2021', '14/10/2021', '21/10/2021', '28/10/2021', '04/11/2021', '11/11/2021', '18/11/2021', '25/11/2021', '02/12/2021',
                     '09/12/2021', '16/12/2021','23/12/2021', '30/12/2021']]
lifespan = lifespan.reset_index(drop=True)
lifespan

(117, 38)
(117, 38)
Index(['Unnamed: 0', 'camera', 'position', 'vial', 'condition',
       'concentration', 'compound', 'repetition', 'replicate', 'date_of_birth',
       'start_flies', 'dont_use', '08/07/2021', '16/07/2021', '22/07/2021',
       '29/07/2021', '05/08/2021', '12/08/2021', '19/08/2021', '26/08/2021',
       '02/09/2021', '09/09/2021', '16/09/2021', '23/09/2021', '30/09/2021',
       '07/10/2021', '14/10/2021', '21/10/2021', '28/10/2021', '04/11/2021',
       '11/11/2021', '18/11/2021', '25/11/2021', '02/12/2021', '09/12/2021',
       '16/12/2021', '23/12/2021', '30/12/2021'],
      dtype='object')


,camera,position,vial,condition,concentration,compound,repetition,replicate,date_of_birth,start_flies,...,28/10/2021,04/11/2021,11/11/2021,18/11/2021,25/11/2021,02/12/2021,09/12/2021,16/12/2021,23/12/2021,30/12/2021
0,1,A,1,SD,NaN,SD,1,A,08/07/2021,10,...,0,0,0,0,0,0,0,0,0,0
1,1,B,2,DMSO-01,0.001%,DMSO,1,A,08/07/2021,10,...,0,0,0,0,0,0,0,0,0,0
2,1,C,3,DMSO-1,0.01%,DMSO,1,A,08/07/2021,10,...,0,0,0,0,0,0,0,0,0,0
3,1,D,4,DMSO-10,0.1%,DMSO,1,A,08/07/2021,10,...,0,0,0,0,0,0,0,0,0,0
4,2,A,5,DMSO-100,1%,DMSO,1,A,09/07/2021,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,9,C,113,A-100,100 µM,A,3,C,18/10/2021,10,...,10,10,10,10,10,6,5,2,0,0
113,9,D,114,B-01,0.1 µM,B,3,C,18/10/2021,10,...,10,10,10,10,10,10,10,9,5,0
114,10,A,115,B-1,1 µM,B,3,C,18/10/2021,10,...,10,10,10,10,10,9,9,9,5,0
115,10,B,116,B-10,10 µM,B,3,C,18/10/2021,10,...,10,10,10,10,9,8,8,4,0,0


# Lifespan data

In [4]:
lifespan_info = lifespan[['condition', 'concentration', 'compound']].drop_duplicates()
lifespan_info.index = lifespan_info['condition']
lifespan_info = lifespan_info.sort_index()
lifespan_info.loc['SD', 'concentration'] = ''
lifespan_info['color'] = np.nan
lifespan_info.loc[lifespan_info['compound'].isin(['A']), 'color'] = 'red'
lifespan_info.loc[lifespan_info['compound'].isin(['B']), 'color'] = 'green'
lifespan_info.loc[lifespan_info['compound'].isin(['DMSO']), 'color'] = 'grey'
lifespan_info.loc[lifespan_info['compound'].isin(['SD']), 'color'] = 'black'
lifespan_info['linewidth'] = np.nan
lifespan_info.loc[lifespan_info['compound'].isin(['SD']), 'linewidth'] = 1
lifespan_info.loc[lifespan_info['concentration'].isin(['0.1 µM', '0.001%']), 'linewidth'] = 1
lifespan_info.loc[lifespan_info['concentration'].isin(['1 µM', '0.01%']), 'linewidth'] = 2
lifespan_info.loc[lifespan_info['concentration'].isin(['10 µM', '0.1%']), 'linewidth'] = 3
lifespan_info.loc[lifespan_info['concentration'].isin(['100 µM', '1%']), 'linewidth'] = 4
lifespan_info['alpha'] = np.nan
lifespan_info.loc[lifespan_info['compound'].isin(['SD']), 'alpha'] = 1
lifespan_info.loc[lifespan_info['compound'].isin(['DMSO']), 'alpha'] = 0.6
lifespan_info.loc[lifespan_info['compound'].isin(['A', 'B']), 'alpha'] = 0.4
lifespan_info['linestyle'] = '-'
lifespan_info.loc[lifespan_info['compound'].isin(['SD']), 'linestyle'] = '--'

# Daily live counts

In [5]:
week_dates = np.array([datetime.strptime(i,'%d/%m/%Y') for i in lifespan.columns[11:]]); print(week_dates.shape)
dob_dates = np.array([datetime.strptime(i,'%d/%m/%Y') for i in lifespan['date_of_birth']]); print(dob_dates.shape)
day_span = (week_dates.max()-dob_dates.min()).days+1; print(day_span)
start_live_counts = lifespan['start_flies'].tolist(); print(len(start_live_counts))

(26,)
(117,)
176
117


In [6]:
weekly_live_counts = np.array(lifespan.iloc[:,11:].to_numpy()); print(weekly_live_counts.shape)
weekly_live_counts_idx = np.array([list(dob_dates[i]>week_dates) for i in range(dob_dates.shape[0])])
weekly_live_counts[weekly_live_counts_idx]=0
weekly_live_counts[0:5,:]

(117, 26)


array([[10, 10,  9,  8,  8,  7,  5,  3,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10,  8,  6,  6,  3,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10,  8,  8,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  9,  9,  7,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [7]:
weekly_day_spans = [i.days for i in (week_dates[1:]-week_dates[:-1])]
daily_live_counts = np.concatenate((np.concatenate(([np.tile(weekly_live_counts[:,i],(weekly_day_spans[i],1)).T for i in range(len(weekly_day_spans))]),axis=1),np.reshape(weekly_live_counts[:,-1],(weekly_live_counts[:,-1].shape[0],1))),axis=1)
daily_live_counts

array([[10, 10, 10, ...,  0,  0,  0],
       [10, 10, 10, ...,  0,  0,  0],
       [10, 10, 10, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  5,  5,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [8]:
for idx in range(daily_live_counts.shape[0]):
    from_idx = (dob_dates[idx]-week_dates.min()).days
    to_idx = min([j for j in [i.days for i in (week_dates-dob_dates[idx])] if j>=0])+(dob_dates[idx]-week_dates.min()).days    
    daily_live_counts[idx,from_idx:to_idx]=start_live_counts[idx]

In [9]:
lifespan['daily_live_counts'] = daily_live_counts.tolist()
lifespan_data = lifespan[['condition', 'daily_live_counts']]
lifespan_data = pd.DataFrame(lifespan_data.groupby(by='condition')['daily_live_counts'].apply(list))
lifespan_data['survs'] = np.array([Survs(np.array(lifespan_data.iloc[i][0])) for i in range(lifespan_data.shape[0])])

In [10]:
def km_fit(surv):
    kmf = KaplanMeierFitter(); kmf.fit(surv['T'], event_observed=surv['E'])
    return(kmf)

def plot_survival_curve(ax, x, y, y1, y2, c, l, lw, a, ls, ci, condition):
    plt.plot(x, y, drawstyle='steps-post', color=c, label=l, linewidth=lw, alpha=a, linestyle=ls)
    
def plot_survival(lifespan_info, lifespan_data, conditions, ci):
    fig, ax = plt.subplots(figsize=(5,5))
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    #ax.set_xticks([]); ax.set_yticks([])
    ax.set_yticks([0, 0.5, 1.0])
    ax.set_xlim([0, 80])
    ax.set_ylim([0, 1.05])
    #plt.xticks(fontsize=0); plt.yticks(fontsize=0)
    plt.xticks(color='w')
    plt.yticks(color='w')

    for condition in conditions:
        print(condition)
        info = lifespan_info.loc[condition]
        #kmf = km_fit(lifespan_data.loc[condition].to_numpy())
        kmf = km_fit((lifespan_data.loc[condition]['survs']))
        plot_survival_curve(ax, 
                            kmf.survival_function_.index, 
                            kmf.survival_function_['KM_estimate'], 
                            kmf.confidence_interval_survival_function_['KM_estimate_lower_0.95'],
                            kmf.confidence_interval_survival_function_['KM_estimate_upper_0.95'],
                            info['color'],
                            (info['compound']+' '+info['concentration']), 
                            info['linewidth'], 
                            info['alpha'],
                            info['linestyle'],
                           ci, condition)
    file_name = str(conditions) +'.png'
    print(file_name)
    plt.savefig(file_name +'.png', dpi=300, transparent=True); plt.close(fig)
plot_survival(lifespan_info, lifespan_data, ['DMSO-01', 'A-01'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-1', 'A-1'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-10', 'A-10'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-100', 'A-100'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-01', 'B-01'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-1', 'B-1'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-10', 'B-10'], False)
plot_survival(lifespan_info, lifespan_data, ['DMSO-100', 'B-100'], False)

DMSO-01
A-01
['DMSO-01', 'A-01'].png
DMSO-1
A-1
['DMSO-1', 'A-1'].png
DMSO-10
A-10
['DMSO-10', 'A-10'].png
DMSO-100
A-100
['DMSO-100', 'A-100'].png
DMSO-01
B-01
['DMSO-01', 'B-01'].png
DMSO-1
B-1
['DMSO-1', 'B-1'].png
DMSO-10
B-10
['DMSO-10', 'B-10'].png
DMSO-100
B-100
['DMSO-100', 'B-100'].png


# LogRank Test

In [12]:
DMSO10=lifespan_data.loc['DMSO-10']
DMSO10=DMSO10['survs']
T_DMSO10=DMSO10['T']
E_DMSO10=DMSO10['E']


B10=lifespan_data.loc['B-10']
B10=B10['survs']
T_B10=B10['T']
E_B10=B10['E']


results=logrank_test(T_DMSO10, T_B10, event_observed_A=E_DMSO10, event_observed_B=E_B10)
results.print_summary()

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic    p  -log2(p)
           1.46 0.23      2.14

In [14]:
logrank_test(T_DMSO100, T_B100, event_observed_DMSOA=E_DMSO100, event_observed_B=E_B100)

<lifelines.StatisticalResult: logrank_test>
                 t_0 = -1
   null_distribution = chi squared
  degrees_of_freedom = 1
event_observed_DMSOA = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
           test_name = logrank_test

---
 test_statistic      p  -log2(p)
          15.49 <0.005     13.55

In [13]:
DMSO100=lifespan_data.loc['DMSO-100']
DMSO100=DMSO100['survs']
T_DMSO100=DMSO100['T']
E_DMSO100=DMSO100['E']


B100=lifespan_data.loc['B-100']
B100=B100['survs']
T_B100=B100['T']
E_B100=B100['E']


results=logrank_test(T_DMSO100, T_B100, event_observed_DMSOA=E_DMSO100, event_observed_B=E_B100)
results.print_summary()

<lifelines.StatisticalResult: logrank_test>
                 t_0 = -1
   null_distribution = chi squared
  degrees_of_freedom = 1
event_observed_DMSOA = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
           test_name = logrank_test

---
 test_statistic      p  -log2(p)
          15.49 <0.005     13.55